In [1]:
%matplotlib inline

import numpy as np
import h5py

import torch
from torch.nn import functional as F
from torch.autograd import Variable

##############################
from models import unet
from datasets import datasets
from utils import manager as mgr
from utils import img_utils
##############################

# set device
#device = torch.device(DEVICE if torch.cuda.is_available() else "cpu")
exp_name = 'unet_df_030_001'
WEIGHTS_PATH = '/home/philipp/Data/weights/'+exp_name+'/'
#device = "cuda:0"
device = "cpu"
print(device)

nr_channels = 7
nr_classes = 4

cpu


In [2]:
## load model with weights

In [3]:
model = unet.UNET(in_channels=nr_channels, out_channels=nr_classes)

try:
    mgr.load_weights(model, WEIGHTS_PATH+'weights-19-0.217-0.821.pth')
    #load_weights(model, WEIGHTS_PATH+'latest_5d.pt')
    print("weights loaded")
except:
    model.apply(mgr.weights_init)
    print("no weights found")
    
model.to(device)

loading weights '/home/philipp/Data/weights/unet_df_030_001/weights-19-0.217-0.821.pth'
loaded weights (lastEpoch 18, loss 0.21727018058300018, error 0.8208388090133667)
weights loaded


UNET(
  (conv1): Sequential(
    (0): Conv2d(7, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256,

In [4]:
## load dataset

In [5]:
class ForestDataset(torch.utils.data.Dataset):

    '''Characterizes a dataset for PyTorch'''

    def __init__(self, path):
        '''Initialization'''
        # open dataset
        self.dset = h5py.File(path, 'r')
        self.ortho = self.dset['ortho']
        self.dsm = self.dset['dsm']
        self.dtm = self.dset['dtm']
        self.slope = self.dset['slope']

        # set number of samples
        self.dataset_size = 10679

        ## TODO:
        # make means and stds load from hdf5
        self.means_tams = np.array([56.12055784563426, 62.130400134006976, 53.03228547781888, 119.50916281232037], dtype='float32')
        self.stds_tams = np.array([30.37628560708646, 30.152693706272483, 23.13718651792004, 49.301477498205074], dtype='float32')

        self.means_dsm = np.array([13.45]).astype(np.float32)
        self.stds_dsm = np.array([10.386633252098674]).astype(np.float32)

        self.means_dtm = np.array([1446.0]).astype(np.float32)
        self.stds_dtm = np.array([271.05322202384195]).astype(np.float32)

        self.means_slope = np.array([22.39]).astype(np.float32)
        self.stds_slope = np.array([11.69830556896441]).astype(np.float32)


    def __len__(self):
        '''Denotes the total number of samples'''
        return self.dataset_size


    def __getitem__(self, index):
        '''Generates one sample of data'''

        # depending on data change mean and std
        means = self.means_tams
        stds = self.stds_tams

        # Load data and get label
        X_ortho = (torch.tensor(self.ortho[index], \
            dtype=torch.float32).permute(2, 0, 1) - \
            means[:, np.newaxis, np.newaxis]) / stds[:, np.newaxis, np.newaxis]
        X_dsm = (torch.tensor(self.dsm[index], \
            dtype=torch.float32).permute(2, 0, 1) - self.means_dsm) / self.stds_dsm
        X_dtm = (torch.tensor(self.dtm[index], \
            dtype=torch.float32).permute(2, 0, 1) - self.means_dtm) / self.stds_dtm
        X_slope = (torch.tensor(self.slope[index], \
            dtype=torch.float32).permute(2, 0, 1) - self.means_slope) / self.stds_slope

        X = torch.cat((X_ortho, X_dsm, X_dtm, X_slope),0)

        return X #torch.from_numpy(y).permute(2, 0, 1)


    def close(self):
        ''' closes the hdf5 file'''
        self.dset.close()


    def show_item(self, index):
        '''shows the data'''
        #plt.imshow(np.array(self.ground_truth[index]))

        fig = plt.figure(figsize=(20,20))

        dic_data = {'RGB' : [np.array(self.ortho[index][:,:,:3]), [0.1, 0.3, 0.5, 0.7]], \
        'CIR' : [np.array(np.roll(self.ortho[index], 1, axis=2)[:,:,:3]), [0.1, 0.3, 0.5, 0.7]], \
        'DSM' : [np.array(self.dsm[index].astype('f')), [10, 20, 30]], \
        'DTM' : [np.array(self.dtm[index].astype('f')), [10, 20, 30]], \
        'Slope' : [np.array(self.slope[index].astype('f')), [10, 20, 30]]}

        for i, key in enumerate(dic_data):
            ax = fig.add_subplot(2, 3, i+1)
            imgplot = plt.imshow(dic_data[key][0])
            ax.set_title(key)
            plt.colorbar(ticks=dic_data[key][1], orientation='horizontal')
            plt.axis('off')

In [6]:
## loading the dataset
#path_dataset = "/home/philipp/Data/dataset_256_df_2.h5"
path_dataset = "/media/philipp/DATA/dataset/dataset_512_df_prediction.h5"
#path_dataset = "/media/philipp/DATA/dataset/dataset_256_df_2.h5"

# open dataset
dataset = ForestDataset(path_dataset)

In [15]:
def quatering(dataset, idx_start, idx_end):
    ## prepare input data
    # cut into 4 quaters = 1 x 7x512x512 -> 4 x 7x256x256
    n = idx_end - idx_start
    store = torch.zeros((n*4,7,256,256),dtype=torch.float32)

    for j,i in enumerate(range(idx_start, idx_end)):
        store[j*4] = dataset[i][:,:256,:256]
        store[j*4+1] = dataset[i][:,256:,:256]
        store[j*4+2] = dataset[i][:,:256,256:]
        store[j*4+3] = dataset[i][:,256:,256:] 
    return store

In [16]:
def predict(store):
    # get predictions
    with torch.no_grad():
        output = model(store)
    pred_256 = mgr.get_predictions(output).numpy()
    return pred_256

In [17]:
def merge(pred_256, idx_start, idx_end):
    ## prepare output data
    # merge into 4 quaters = 4 x 256x256 -> 1 x 512x512
    n = idx_end - idx_start
    pred_512 = np.zeros((n,512,512), dtype=np.int8)
    
    for i in range(n):
        j = i*4
        pred_512[i,:256,:256] = pred_256[j]
        pred_512[i,256:,:256] = pred_256[j+1]
        pred_512[i,:256,256:] = pred_256[j+2]
        pred_512[i,256:,256:] = pred_256[j+3]
    return pred_512    

In [18]:
start = 0
end = 160
batch = 40

p = np.zeros((end-start,512,512), dtype=np.int8)
print(p.shape)

counter = start

for i in range((end-start) // batch):
    
    store = print(counter, counter+batch)
    
    store = quatering(dataset=dataset, idx_start=counter, idx_end=counter+batch)
    pred_256 = predict(store)
    pred_512 = merge(pred_256, counter, counter+batch)
    
    p[i*batch:(i+1)*batch] = pred_512
    
    counter += batch
    


(160, 512, 512)
0 40
40 80
80 120
120 160


In [20]:
p.shape

(160, 512, 512)

In [21]:
np.save('pred.npy', p)

In [ ]:
##################################################################

In [9]:
## prepare input data
# cut into 4 quaters = 1 x 7x512x512 -> 4 x 7x256x256

start = 0
end = 40

n = end - start
store = torch.zeros((n*4,7,256,256),dtype=torch.float32)

for i,j in enumerate(range(0,n*4,4)):
    store[j] = dataset[i][:,:256,:256]
    store[j+1] = dataset[i][:,256:,:256]
    store[j+2] = dataset[i][:,:256,256:]
    store[j+3] = dataset[i][:,256:,256:]

In [10]:
# get predictions
with torch.no_grad():
    output = model(store)
p = mgr.get_predictions(output).numpy()

In [11]:
p.shape

(160, 256, 256)

In [12]:
## prepare output data
# merge into 4 quaters = 4 x 7x256x256 -> 1 x 7x512x512

pred = np.zeros((n,512,512), dtype=np.int8)

for i,j in enumerate(range(0,n*4,4)):
    pred[i,:256,:256] = p[j]
    pred[i,256:,:256] = p[j+1]
    pred[i,:256,256:] = p[j+2]
    pred[i,256:,256:] = p[j+3]

In [13]:
pred.shape

(40, 512, 512)

In [12]:
mean = 119.50916281232037
std = 49.301477498205074

i = 0
l = 10000
nirs = []

for i in range(20):
    # grab data
    x = torch.stack([dataset[i][3], \
                    dataset[i+1*l][3], \
                    dataset[i+2*l][3], \
                    dataset[i+3*l][3]])
    
    # reconstruct tile
    merged_512 = np.ones((512, 512))
    # paste predicted data into array
    merged_512[:256,:256] = dataset[i][3]
    merged_512[256:,:256] = dataset[i+1*l][3]
    merged_512[:256,256:] = dataset[i+2*l][3]
    merged_512[256:,256:] = dataset[i+3*l][3]
    
    merged_512 = std * merged_512 + mean
    
    nirs.append(merged_512)

In [13]:
nirs = np.array(nirs)

In [15]:
np.save('nir.npy', nirs)

In [16]:
np.save('pred.npy', p)

In [ ]:
## convert to geotiff

In [1]:
import os
import numpy as np

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

In [2]:
def array2raster(newRasterfn, dataset, array, dtype):
    """
    save GTiff file from numpy.array
    input:
        newRasterfn: save file name
        dataset : original tif file
        array : numpy.array
        dtype: Byte or Float32.
    """
    cols = array.shape[1]
    rows = array.shape[0]
    originX, pixelWidth, b, originY, d, pixelHeight = dataset.GetGeoTransform() 

    driver = gdal.GetDriverByName('GTiff')

    # set data type to save.
    GDT_dtype = gdal.GDT_Unknown
    if dtype == "Byte": 
        GDT_dtype = gdal.GDT_Byte
    elif dtype == "Float32":
        GDT_dtype = gdal.GDT_Float32
    else:
        print("Not supported data type.")

    # set number of band.
    if array.ndim == 2:
        band_num = 1
    else:
        band_num = array.shape[2]

    outRaster = driver.Create(newRasterfn, cols, rows, band_num, GDT_dtype)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))

    # Loop over all bands.
    for b in range(band_num):
        outband = outRaster.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        if band_num == 1:
            outband.WriteArray(array)
        else:
            outband.WriteArray(array[:,:,b])

    # setteing srs from input tif file.
    prj=dataset.GetProjection()
    outRasterSRS = osr.SpatialReference(wkt=prj)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

In [3]:
pred = np.load('pred.npy')

In [4]:
path_input = ['/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_121889.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_121890.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_122277.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_122278.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_122279.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_122280.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_122647.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_122648.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_122649.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_122650.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_122651.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_122652.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_122653.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123009.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123010.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123011.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123012.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123013.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123014.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123015.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123016.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123017.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123349.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123350.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123351.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123352.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123353.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123354.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123355.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123356.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123357.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123658.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123659.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123660.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123661.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123662.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123663.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123664.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123949.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123950.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123951.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_123952.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124241.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124242.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124243.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124515.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124516.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124517.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124518.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124519.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124520.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124521.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124796.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124797.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124798.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124799.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124800.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124801.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124802.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124803.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124804.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124805.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124806.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_124807.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125073.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125074.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125075.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125076.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125077.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125078.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125079.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125080.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125081.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125082.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125368.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125369.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125370.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125371.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125372.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125373.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125374.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125375.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125671.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125672.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125673.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125953.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125954.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125955.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125956.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125957.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_125958.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_126242.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_126243.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_126244.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_126245.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_126246.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_126554.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_126555.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_126556.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_126557.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_126858.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_126859.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_126860.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_126861.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_127151.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_127152.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_127153.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_127154.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_127433.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_127434.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_127435.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_127436.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_127437.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_127438.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_127710.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_127711.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_127968.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_128223.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_128224.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_128225.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_128226.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_128474.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_128475.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_128476.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_128703.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_128704.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129204.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129205.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129455.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129456.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129457.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129458.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129704.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129705.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129706.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129707.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129708.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129957.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129958.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129959.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129960.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_129961.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130190.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130191.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130192.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130193.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130194.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130195.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130196.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130427.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130428.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130644.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130645.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130646.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130647.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130862.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130863.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130864.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130865.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130866.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_130867.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131076.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131077.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131078.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131079.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131080.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131081.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131082.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131083.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131283.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131284.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131285.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131286.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131287.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131288.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131289.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131290.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131291.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131292.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131293.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131481.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131482.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131483.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131484.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131485.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131486.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131487.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131488.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131489.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131490.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131687.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131688.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131689.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131690.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131691.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131692.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131693.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131694.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131695.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131696.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131890.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131891.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131892.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131893.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131894.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131895.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131896.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131897.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131898.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131899.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131900.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131901.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_131902.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132094.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132095.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132096.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132097.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132098.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132099.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132100.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132101.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132102.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132103.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132104.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132105.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132106.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132300.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132301.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132302.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132303.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132304.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132305.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132306.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132307.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132308.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132309.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132310.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132311.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132312.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132500.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132501.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132502.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132503.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132504.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132505.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132506.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132507.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132508.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132509.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132510.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132511.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132689.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132690.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132691.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132692.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132693.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132694.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132695.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132696.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132872.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132873.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132874.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132875.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132876.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_132877.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133055.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133056.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133057.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133058.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133059.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133060.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133061.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133254.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133255.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133256.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133257.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133258.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133259.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133260.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133261.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133471.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133472.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133473.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133474.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133475.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133476.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133477.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133478.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133479.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133693.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133694.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133695.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133696.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133697.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133698.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133699.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133700.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133701.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133702.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133703.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133704.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133932.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133933.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133934.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133935.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133936.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133937.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133938.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133939.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133940.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133941.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133942.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133943.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133944.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133945.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133946.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133947.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133948.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_133949.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134176.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134177.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134178.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134179.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134180.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134181.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134182.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134183.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134184.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134185.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134186.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134187.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134188.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134189.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134190.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134191.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134417.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134418.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134419.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134420.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134421.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134422.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134423.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134424.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134425.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134650.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134651.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134652.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134653.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134654.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134655.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134656.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134657.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134879.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134880.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134881.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134882.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134883.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134884.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134885.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134886.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134887.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_134888.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135111.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135112.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135113.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135114.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135115.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135116.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135117.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135347.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135348.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135349.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135350.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135351.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135352.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135353.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135354.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135355.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135356.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135585.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135586.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135587.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135588.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135589.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135590.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135591.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135592.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135593.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135594.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135595.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135834.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135835.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135836.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135837.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135838.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135839.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135840.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135841.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135842.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135843.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_135844.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136091.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136092.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136093.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136094.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136095.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136096.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136097.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136098.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136354.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136355.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136356.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136357.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136358.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136359.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136360.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136361.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136362.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136363.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136364.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136619.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136620.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136621.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136622.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136623.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136624.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136625.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136626.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136627.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136628.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136629.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136630.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136631.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136632.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136633.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136634.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136635.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136636.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136864.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136865.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136866.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136867.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136868.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136869.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136870.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136871.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136872.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136873.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136874.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136875.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136876.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_136877.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137080.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137081.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137082.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137083.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137084.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137085.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137086.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137087.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137088.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137290.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137291.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137292.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137293.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137294.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137295.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137296.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137297.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137298.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137502.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137503.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137504.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137505.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137506.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137507.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137508.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137509.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137510.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137511.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137512.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137718.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137719.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137720.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137721.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137722.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137723.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137724.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137725.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137726.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137727.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137728.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137913.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137914.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137915.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137916.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137917.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137918.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137919.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137920.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137921.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137922.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_137923.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138115.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138116.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138117.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138118.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138119.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138120.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138121.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138122.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138123.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138124.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138298.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138299.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138300.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138301.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138302.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138303.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138304.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138305.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138464.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138465.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138466.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138467.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138468.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138469.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138470.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138471.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138626.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138627.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138628.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138629.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138630.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138631.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138632.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138633.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138634.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138793.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138794.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138795.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138796.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138797.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138798.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138799.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138800.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138973.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138974.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138975.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138976.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138977.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138978.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138979.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138980.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138981.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_138982.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139152.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139153.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139154.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139155.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139156.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139157.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139158.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139159.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139160.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139161.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139162.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139322.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139323.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139324.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139325.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139326.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139327.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139328.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139329.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139330.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139331.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139332.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139333.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139507.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139508.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139509.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139510.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139511.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139512.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139513.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139514.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139515.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139516.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139517.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139518.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139519.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139520.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139521.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139522.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139705.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139706.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139707.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139708.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139709.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139710.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139711.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139712.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139713.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139714.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139715.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139716.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139717.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139718.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139719.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139720.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139899.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139900.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139901.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139902.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139903.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139904.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139905.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139906.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139907.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139908.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139909.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139910.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139911.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139912.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139913.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139914.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_139915.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140091.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140092.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140093.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140094.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140095.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140096.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140097.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140098.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140099.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140100.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140101.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140102.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140103.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140104.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140105.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140106.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140107.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140298.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140299.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140300.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140301.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140302.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140303.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140304.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140305.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140306.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140307.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140308.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140309.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140310.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140311.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140312.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140313.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140314.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140504.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140505.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140506.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140507.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140508.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140509.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140510.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140511.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140512.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140513.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140514.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140515.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140516.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140517.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140518.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140693.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140694.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140695.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140696.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140697.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140698.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140699.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140700.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140701.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140702.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140703.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140704.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140886.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140887.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140888.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140889.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140890.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140891.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140892.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140893.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140894.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140895.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140896.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140897.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_140898.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141081.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141082.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141083.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141084.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141085.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141086.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141087.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141088.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141089.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141090.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141289.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141290.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141291.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141292.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141293.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141294.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141295.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141296.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141297.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141298.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141487.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141488.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141489.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141490.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141491.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141492.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141493.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141494.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141665.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141666.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141667.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141668.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141669.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141670.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141671.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141672.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141673.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141834.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141835.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141836.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141837.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141838.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141839.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141840.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141841.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_141842.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142000.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142001.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142002.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142003.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142004.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142005.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142006.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142007.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142164.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142165.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142166.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142167.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142168.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142169.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142317.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142318.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142319.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142460.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142461.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142462.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142463.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142614.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142615.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142616.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142617.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142618.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142619.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142620.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142778.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142779.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142780.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142781.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142782.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142783.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142784.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142785.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142786.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142787.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142788.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142946.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142947.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142948.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142949.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142950.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142951.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142952.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142953.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142954.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142955.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_142956.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143114.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143115.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143116.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143117.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143118.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143119.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143120.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143121.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143122.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143123.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143124.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143125.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143285.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143286.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143287.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143288.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143289.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143290.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143291.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143292.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143293.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143294.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143295.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143296.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143297.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143298.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143299.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143470.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143471.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143472.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143473.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143474.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143475.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143476.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143477.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143478.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143479.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143480.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143481.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143482.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143645.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143646.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143647.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143648.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143649.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143650.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143651.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143652.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143653.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143654.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143803.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143804.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143805.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143806.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143807.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143808.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143809.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143960.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143961.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143962.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143963.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143964.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143965.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143966.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143967.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143968.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_143969.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144112.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144113.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144114.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144115.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144116.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144117.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144118.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144119.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144120.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144121.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144263.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144264.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144265.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144266.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144267.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144268.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144269.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144270.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144271.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144272.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144273.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144416.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144417.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144418.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144419.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144420.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144421.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144422.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144423.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144424.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144425.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144426.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144427.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144428.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144429.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144430.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144431.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144585.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144586.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144587.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144588.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144589.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144590.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144591.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144592.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144593.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144594.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144595.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144596.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144597.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144598.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144599.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144600.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144601.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144602.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144761.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144762.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144763.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144764.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144765.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144766.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144767.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144768.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144769.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144770.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144771.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144772.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144773.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144774.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144775.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144776.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144777.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144778.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144935.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144936.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144937.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144938.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144939.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144940.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144941.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144942.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144943.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144944.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144945.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144946.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144947.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144948.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144949.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144950.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144951.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144952.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144953.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_144954.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145095.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145096.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145097.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145098.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145099.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145100.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145101.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145102.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145103.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145104.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145105.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145106.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145107.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145108.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145109.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145110.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145111.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145112.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145113.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145114.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145115.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145256.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145257.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145258.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145259.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145260.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145261.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145262.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145263.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145264.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145265.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145266.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145267.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145268.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145269.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145270.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145271.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145272.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145273.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145274.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145275.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_145420.tif']

In [5]:
len(path_input)

1000

In [6]:
pred.shape[0]

160

In [7]:
for i, path_in in enumerate(path_input[:pred.shape[0]]):
    path_out = 'foto/pred_{}.tif'.format(i)
    dataset = gdal.Open(path_in, gdal.GA_ReadOnly)
    array2raster(newRasterfn=path_out, dataset=dataset, array=pred[i], dtype='Byte')